<a href="https://colab.research.google.com/github/pabloac31/TFG/blob/master/adversarial_attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/pabloac31/TFG.git
%cd TFG

Cloning into 'TFG'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 41 (delta 6), reused 32 (delta 4), pack-reused 0
Unpacking objects: 100% (41/41), done.
/content/TFG


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
import torch
import torchvision

import matplotlib.pyplot as plt
import numpy as np

from cifar10_models import *

import torchvision.transforms as transforms

from tqdm import tqdm as pbar

In [0]:
iv3 = inception_v3(pretrained=True)

In [0]:
def make_dataloaders(params):
    """
    Make a Pytorch dataloader object that can be used for valiation
    Input:
        - params dict with key 'path' (string): path of the dataset folder
        - params dict with key 'batch_size' (int): mini-batch size
    Output:
        - testloader (pytorch dataloader object)
    """
    
    transform_validation = transforms.Compose([transforms.ToTensor(),
                                               transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    
    #transform_validation = transforms.Compose([transforms.ToTensor()])
    
    testset = torchvision.datasets.CIFAR10(root=params['path'], train=False, transform=transform_validation, download=True)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=params['batch_size'], shuffle=False, num_workers=4)
    return testloader

In [30]:
data_params = {'path': './data', 'batch_size': 256}
test_loader = make_dataloaders(data_params)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
def test_model(model, params):
    
    model = model.to(params['device']).eval()     
    phase = 'validation'
    logs = {'Accuracy': 0.0}
            
    # Iterate over data
    for image, label in pbar(params[phase+'_loader']):
        image = image.to(params['device'])
        label = label.to(params['device'])

        with torch.no_grad():
            prediction = model(image)
            accuracy = torch.sum(torch.max(prediction, 1)[1] == label.data).item()
            logs['Accuracy'] += accuracy
            
    logs['Accuracy'] /= len(params[phase+'_loader'].dataset)
    
    return logs['Accuracy']

In [34]:
# Train on cuda if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using", device)

test_params = {'description': 'Test', 'device': device, 'validation_loader': test_loader}
test_model(iv3, test_params)

  0%|          | 0/40 [00:00<?, ?it/s]

Using cuda:0


100%|██████████| 40/40 [00:12<00:00,  3.20it/s]


0.9541